# Fast Wavelet Transformation (FWT) Implementation

In [1]:
import numpy as np
from math import log, ceil, floor
import pywt   

In [2]:
def nextPowerOf2(n): 
    count = 0; 
    if (n and not(n & (n - 1))): 
        return n       
    while( n != 0): 
        n >>= 1
        count += 1     
    return 1 << count;  

### Haar One Level

In [3]:
def haarOneLevel(g):
    L = int(len(g)/2)
    G_low, G_high = np.zeros(L), np.zeros(L)
    for i in range(L):
        v0, v1 = g[2*i], g[2*i+1]
        G_low[i] = (v0 + v1)/np.sqrt(2.0)
        G_high[i] = (v0 - v1)/np.sqrt(2.0)
    return G_low, G_high

def haarOneLevelInverse(G):
    L = int(len(G))
    g = np.zeros((L))
    for i in range(L//2):
        v0, v1 = G[i], G[i+int(L/2)]
        g[2*i] = (v0 + v1)/np.sqrt(2.0)
        g[2*i+1] = (v0 - v1)/np.sqrt(2.0)
    return g

### Daubechies-4 (D4) Tap

In [4]:
def db4(g):
    L = int(len(g))
    l0 = (1+np.sqrt(3))/(4*np.sqrt(2))
    l1 = (3+np.sqrt(3))/(4*np.sqrt(2))
    l2 = (3-np.sqrt(3))/(4*np.sqrt(2))
    l3 = (1-np.sqrt(3))/(4*np.sqrt(2))
    h0, h1, h2, h3 = l3, -l2, l1, -l0

    G_low, G_high = np.zeros(L//2), np.zeros(L//2)
    for i in range(0, len(g), 2):
        G_low[i//2] = g[i]*l0 + g[i+1]*l1 + g[(i+2)%L]*l2 + g[(i+3)%L]*l3
        G_high[i//2] = g[i]*h0 + g[i+1]*h1 + g[(i+2)%L]*h2 + g[(i+3)%L]*h3
   
    return G_low, G_high  

def db4Inverse(G):
    L = int(len(G))
    l0 = (1+np.sqrt(3))/(4*np.sqrt(2))
    l1 = (3+np.sqrt(3))/(4*np.sqrt(2))
    l2 = (3-np.sqrt(3))/(4*np.sqrt(2))
    l3 = (1-np.sqrt(3))/(4*np.sqrt(2))
    h0, h1, h2, h3 = l3, -l2, l1, -l0

    g = np.zeros((L))
    for i in range(L//2):
        g[2*i] = G[i-2]*l2 + G[i-2+L//2]*h2 + G[i-1]*l0 + G[i-1+L//2]*h0
        g[2*i+1] = G[i-2]*l3 + G[i-2+L//2]*h3 + G[i-1]*l1 + G[i-1+L//2]*h1
   
    return g

fwt implementation using recursion

In [5]:
def fwt(g, mode='haar'):
    if mode == 'haar':
        G_low, G_high = haarOneLevel(g)
        if len(g) == 2:
            return np.concatenate((G_low, G_high))
        else:
            return np.concatenate((fwt(G_low), G_high))  
    
    if mode == 'db4':
        G_low, G_high = db4(g)
        if len(g) == 4:
            return np.concatenate((G_low, G_high))
        else:
            return np.concatenate((fwt(G_low, 'db4'), G_high)) 

In [6]:
def ifwt(G, mode='haar'):
    if mode == 'haar':
        L = int(log(len(G),2))
        first_half = G[:2**1]
        for i in range(1, L):
            first_half = haarOneLevelInverse(first_half)
            second_half = G[2**i:2**(i+1)]   
            first_half = np.concatenate((first_half, second_half))
        return haarOneLevelInverse(first_half)
    
    if mode == 'db4':
        L = int(log(len(G),2))-1
        first_half = G[:4**1]
        for i in range(1, L):
            first_half = db4Inverse(first_half)
            second_half = G[4**i:4**(i+1)]   
            first_half = np.concatenate((first_half, second_half))
        return db4Inverse(first_half)

In [7]:
# Function to check 
# Log base 2 
def Log2(x): 
    if x == 0: 
        return false; 
  
    return (log(x) / log(2)); 
  
# Function to check 
# if x is power of 2 
def isPowerOfTwo(n): 
    return (ceil(Log2(n)) == floor(Log2(n)))

### Demo - Power of 2 (Haar)

In [8]:
g = np.array([1,7,3,0,5,4,2,9])
G = fwt(g)
G

array([10.96015511, -3.18198052,  2.5       , -1.        , -4.24264069,
        2.12132034,  0.70710678, -4.94974747])

In [9]:
ifwt(G)

array([ 1.00000000e+00,  7.00000000e+00,  3.00000000e+00, -3.14018492e-16,
        5.00000000e+00,  4.00000000e+00,  2.00000000e+00,  9.00000000e+00])

### Demo - Nearest Power of 2 (Haar)

In [10]:
g = np.array([1,7,3,0,5,2,0])
L = len(g)
if isPowerOfTwo(L) == False:
    num_pads = nextPowerOf2(L) - L
    g = np.concatenate((g, np.zeros(num_pads)))
else:
    num_pads = 0

In [11]:
G = fwt(g)
G[:] if num_pads == 0 else G[:-num_pads]

array([ 6.36396103,  1.41421356,  2.5       ,  3.5       , -4.24264069,
        2.12132034,  2.12132034])

In [12]:
g = ifwt(G)
g[:-num_pads]

array([ 1.00000000e+00,  7.00000000e+00,  3.00000000e+00, -6.28036983e-16,
        5.00000000e+00,  2.00000000e+00, -4.44089210e-16])

In [13]:
cA, cD = pywt.dwt([1,7,3,0,5,2,0], 'haar')
print(cA, cD)

[5.65685425 2.12132034 4.94974747 0.        ] [-4.24264069  2.12132034  2.12132034  0.        ]


In [14]:
cA, cD = pywt.dwt([1,7,3,0,5,2,0], 'haar')
print(cA, cD)

[5.65685425 2.12132034 4.94974747 0.        ] [-4.24264069  2.12132034  2.12132034  0.        ]


### Demo - Nearest Power of 2 (D4)

In [15]:
g = np.array([1,1,4,4,0,0,1,1])
L = len(g)
if isPowerOfTwo(L) == False:
    num_pads = nextPowerOf2(L) - L
    g = np.concatenate((g, np.zeros(num_pads)))
else:
    num_pads = 0

In [16]:
G = fwt(g, 'db4')
G[:] if num_pads == 0 else G[:-num_pads]

array([ 5.07355716e+00,  9.26442841e-01, -2.00656986e+00, -1.45753175e+00,
        1.06066017e+00, -1.41421356e+00,  3.53553391e-01, -1.11022302e-16])

In [17]:
g = ifwt(G, 'db4')
g[:] if num_pads == 0 else g[:-num_pads]

array([ 2.1472754 , -2.56518375,  5.00739109,  0.10518829, -0.01105716,
       -3.60584844, -0.51428313,  3.71388157])